---
# 📁 Hyun's Code collection (BeautifulSoup) 
---

### <h3 align="right">🥇 Authored by <strong>Hyun</strong></h3>

# ✏️  What is **BeautifulSoup**?

- Web에 있는 데이터를 가져와서 저장할 수 있는 패키지
- 크롤링할 때 가장 많이 쓰는 라이브러리
- Selenium보다 에러가 적으며 속도가 빠르다.

# ✏️ Import Libraries

In [ ]:
# Libraries for Crawling
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# ✏️  Methods and Examples

- **requests .get( " 주소 " , headers = {' User-Agent ' : ' Mozilla/5.0 ' } )**: URL 주소에 접속을 요청해 여러가지 데이터를 가져옴
  - header를 달아주면 anti-crawling을 피해갈 수 있음
  - 깐깐한 사이트는 user agent를 전부 다 입력해야 하는 경우도 있어서 그때는 https://developers.whatismybrowser.com/useragents/explore/software_name/chrome/ 여기에 들어가서 전체를 복사해야 함

  **.text**: 접속한 URL의 소스코드

  **.elapsed**: 페이지가 응답하는데 걸린 시간
- **BeautifulSoup ( 소스코드, 'html .parser ' )**: BeautifulSoup 함수는 선택자를 통하여 데이터를 선택할 수 있도록 hyml 소스코드를 태그 기준으로 파싱해줌

  **.select ( " 데이터 위치 " )**: 선택자를 사용해서 데이터를 골라줌. 모든 데이터를 list로 저장한다.
  
  **.select_one ( " 데이터 위치 " )**: 선택자에 해당하는 데이터 중 첫번째 데이터를 가져옴
  
  **.text**: 선택한 데이터를 텍스트 형식으로 바꿔줌

### 📔 Examples (Crawling Using URL address)

In [ ]:
# Crawl bills contents 
dict_crawl_data = {}
for idx in tqdm(sug.index):
    bill_id = sug.loc[idx]['BILL_ID']
    url = sug.loc[idx]['DETAIL_LINK']

    res = requests.get(url)

    soup = BeautifulSoup(res.text, 'html.parser')
    
    content = soup.select('#summaryContentDiv')[0].text.replace('\n', '').replace('\t', '').replace('\r', '')
    dict_crawl_data[bill_id] = content

### 📔 Examples (Crawling Search Results)

In [ ]:
# 예시
# 다음 '코알라' 검색 결과 crawling
#1.엑셀 저장
import requests
from bs4 import BeautifulSoup

import openpyxl

wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["제목","내용"])

for p in range(1,3):
    raw = requests.get("https://search.daum.net/search?w=news&q=코알라&p="+str(p))
    html = BeautifulSoup(raw.text, "html.parser")
    cons = html.select("div.wrap_cont")


    for con in cons:
        title = con.select_one("a.f_link_b").text.strip()
        ab = con.select_one("p.f_eb.desc").text.strip()
        sheet.append([title, ab])

wb.save("코알라news.xlsx")

#2.CSV저장
f = open("코알라news.csv","w", encoding='utf-8')
f.write("제목, 내용\n")

for p in range(1,3):
    raw = requests.get("https://search.daum.net/search?w=news&q=코알라&p="+str(p)).text
    html = BeautifulSoup(raw, "html.parser")
    cons = html.select("div.wrap_cont")

    for con in cons:
        title = con.select_one("a.f_link_b").text
        ab = con.select_one("p.f_eb.desc").text
        #print(title, ab)
        title = title.replace(",", "")
        ab = ab.replace(",", "")
        f.write(title+","+ab+"\n")

f.close()

### 📔 Examples (Crawling Daum Movies)

In [ ]:
# 예시
# 다음 영화 crawling
import requests
from bs4 import BeautifulSoup
import openpyxl

try:
    # 워크북 불러오기, 현재 활성화된 시트 선택하기
    wb = openpyxl.load_workbook("week5_hw_1.xlsx")
    sheet = wb.active
    print("불러오기 완료")
# 파일 불러오기에 실패하면, 새로운 워크북(엑셀파일)을 만듭니다.
except:
    # 워크북 새로 만들기, 현재 활성화된 시트 선택하기
    # 헤더 행 추가하기
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["제목", "평점", "장르", "감독", "배우"])
    print("새로운 파일을 만들었습니다.")
    
raw = requests.get("http://ticket2.movie.daum.net/Movie/MovieRankList.aspx",
                   headers={'User-Agent':'Mozilla/5.0'})
html = BeautifulSoup(raw.text, "html.parser")

movies = html.select("ul.list_boxthumb li")
num=0
for movie in movies:
    num = num+1
    title = movie.select_one("div strong a.link_g")
    url = title.attrs["href"]
    
    raw_each_movie = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    html_each_movie = BeautifulSoup(raw_each_movie.text, 'html.parser')
    
    contents = html_each_movie.select('div.detail_summarize')
    try:
        title = html_each_movie.select_one('div.subject_movie strong.tit_movie')
        star = html_each_movie.select_one('em.emph_grade')
        genre = html_each_movie.select_one('dl.list_movie > dd:nth-of-type(1)')
        director = html_each_movie.select_one('dl.list_movie > dd:nth-of-type(5) a')
        actor = html_each_movie.select_one('dl.list_movie > dd:nth-of-type(6) a')
        sheet.append([title.text, star.text, genre.text, director.text, actor.text])
        print("{}번째 끝!".format(num))
    except:
        continue
wb.save("week5_hw_1.xlsx")

### 📔 Examples (Crawling Job site)

In [ ]:
# 팜리쿠르트 크롤링 코드
class Crawler:
    def __init__(self):
        self.name = "I am a good robot"

    def load_past_data(self, url):
        with open(url,'r') as f:
            data = json.load(f)
        return data

    def crawl_data(self):
        url = 'http://recruit.dailypharm.com/Search.php?mode=offer&order=reg&optionAreaVal%5B%5D=97&optionAreaVal%5B%5D=100&optionAreaVal%5B%5D=104&optionAreaVal%5B%5D=106&optionAreaVal%5B%5D=107&optionAreaVal%5B%5D=109&optionAreaVal%5B%5D=110&optionAreaVal%5B%5D=111&optionJobVal%5B%5D=17&optionJobVal%5B%5D=12&optionJobVal%5B%5D=13&keyword='
        res = requests.get(url)

        soup = BeautifulSoup(res.text, 'html.parser')
        contents = soup.select("li.search_tabCont_wrap a")

        dict_info = {}
        for content in contents:
            dict_temp = {}
            href = content['href']
            content_url = 'http://recruit.dailypharm.com' + href
            id_ = content_url.split('.php?ID=')[1]
            
            # crawl title and region
            res_2 = requests.get(content_url)
            soup_2 = BeautifulSoup(res_2.content.decode('euc-kr','replace'))
            title = soup_2.select("div.offer_title_wrap h2")[0].text
            region = soup_2.select("div.boxList td.boxbody")[4].text.strip()
            
            # crawl details
            res_3 = requests.get(content_url.replace('OfferView', 'offerContens'))
            soup_3 = BeautifulSoup(res_3.content.decode('euc-kr','replace'))
            details = ''
            for i in soup_3.select("body p"):
                details = details + '\n' + i.text.strip()
            
            # save crawled informations
            dict_temp['title'] = title
            dict_temp['region'] = region
            dict_temp['details'] = details
            dict_info[id_] = dict_temp
        return dict_infohtml = BeautifulSoup(raw.text, 'html.parser')
print(html)

clips = html.select("div.inner")
print(clips[0])

title = clips[0].select_one("dt.title")
print(title)

print(title.text)    # 선택한 데이터를 텍스트 형식으로 바꿔줌

### 📔 Examples (Crawling by Pages)

In [ ]:
# 페이지 넘어가기
for n in range(1, 100, 10):
  raw = requests.get("https://search.naver.com/search.naver?where=news*query=코알라&start="+str(n), 
                  headers={'User-Agent':'Mozilla/5.0'})
  html = BeautifulSoup(raw.text, "html.parser")
  
  articles = html.select("ul.type01 > li")
  
  for article in articles:
    title = article.select_one("a._sp_each_title").text
    source = article.select_one("span._sp_each_source").text
    
    print( title, source)